In [ ]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, r2_score

# Cargar datos
df2 = pd.read_csv('/mnt/data/datos2_limpios.csv')

df2_limpio = df2[['Edad', 'Ingreso Anual (€)', 'Porcentaje Ahorro (%)',
                   'Ahorro Actual (€)', 'Gasto Mensual (€)', 'Vida Esperada (años)',
                   'Años Hasta Jubilación']]

# Definir variables de entrada y salida
X = df2_limpio.drop(columns=['Años Hasta Jubilación'])
y = df2_limpio['Años Hasta Jubilación']

# Dividir en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Modelo con Random Forest y búsqueda de hiperparámetros
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}

rf = RandomForestRegressor(random_state=0)
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_

# Evaluar modelo
y_pred = best_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'MAE: {mae}, R2: {r2}')

# Cargar nuevo dataset y predecir
df = pd.read_csv('/mnt/data/datos1_limpios.csv')
X_2 = df[['Edad', 'Ingreso Anual (€)', 'Porcentaje Ahorro (%)',
          'Ahorro Actual (€)', 'Gasto Mensual (€)', 'Vida Esperada (años)']]

# Asegurar que no haya valores nulos o inconsistentes
X_2 = X_2.dropna()

# Predecir
predicciones = best_model.predict(X_2)
df.loc[X_2.index, 'Años Hasta Jubilación'] = predicciones
df.loc[X_2.index, 'Edad de Jubilación'] = df.loc[X_2.index, 'Edad'] + df.loc[X_2.index, 'Años Hasta Jubilación']

# Mostrar resultados
df.head()